In [52]:
import pandas as pd
Data = pd.read_csv('C:/Users/muneeba/Desktop/THESIS/DataDistributionPCA.csv')
Data=Data.sort_values(['Sensor ID'],ascending=[True])
Data1=Data.iloc[:,4:68]
Data1.head()
returns = Data1[[key for key in dict(Data1.dtypes) if dict(Data1.dtypes)[key] in ['float64', 'int64']]] 
returns = returns[:64]
returns.head()
#slicing the required data out of complete dataset
returns_arr = returns.as_matrix()[:64]
returns_arr.shape
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
returns_arr_scaled = scaler.fit_transform(returns_arr)
#logdir = r'C:/Users/muneeba/Desktop/THESIS/logdir'

Set up the TensorFlow code for an Autoencoder
The number of inputs of the auto-encoder is equal to the number of outputs (number of columns in our input data)
We have one hidden layer with 3 neurons
Use an AdamOptimizer to minimize the loss
Notice that:
The reconstruction loss of the output is the mean-square error
Each layer has no activation function i.e is a linear layer
An Autoencoder set up this way simply performs a PCA

In [53]:
import tensorflow as tf

n_inputs = 64
n_hidden = 3  # codings
n_outputs = n_inputs

learning_rate = 0.01 #how much to train 
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden = tf.layers.dense(X, n_hidden)
outputs = tf.layers.dense(hidden, n_outputs)

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(reconstruction_loss)

init = tf.global_variables_initializer()




Run the training data through the Autoencoder
We do not use any labels for this training only the input data
The output is the reconstructed value using only the 3 principal components
Note that the reconstruction will not be equal to the input data

In [54]:
n_iterations = 10000

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={X: returns_arr_scaled})
    writer = tf.summary.FileWriter('./temp', sess.graph)
    outputs_val = outputs.eval(feed_dict={X: returns_arr_scaled})
    print(outputs_val)

[[ 0.98170203  0.99483538  1.0264951  ...,  0.80608475  0.62092239
   0.43926886]
 [ 0.68050206  0.69133967  0.71772844 ...,  0.75056088  0.60264355
   0.45749852]
 [-0.3122336  -0.30895546 -0.2999765  ...,  0.57056665  0.54477894
   0.51941514]
 ..., 
 [-0.21985134 -0.21630879 -0.19744173 ...,  0.16433796  0.22465159
   0.27242345]
 [ 0.9832322   0.99500418  1.05217648 ..., -0.50169253 -0.38587821
  -0.30726635]
 [ 1.19695795  1.21011639  1.27378035 ..., -0.6235131  -0.49818021
  -0.41427737]]


In [55]:
import numpy as np
data=scaler.inverse_transform(outputs_val,copy=None)
#recovered
#data
orig=np.linalg.norm(outputs_val,ord=1)
DiffData=data-returns_arr
NewNorm=np.linalg.norm(DiffData,ord=1)
error=NewNorm/orig 
error
#recovering the orignal dat

0.00018285232327918362

In [56]:
#building stacked autoencoders
n_inputs = 64
n_hidden1 = 14
n_hidden2 = 3 # codings
n_hidden3 = n_hidden1
n_outputs = n_inputs

In [57]:
he_init = tf.contrib.layers.variance_scaling_initializer()
l2_regularizer = tf.contrib.layers.l2_regularizer(0.0001)

In [58]:
X = tf.placeholder(tf.float32, shape=[None, n_inputs])

In [59]:
from functools import partial

dense_layer = partial(tf.layers.dense,
                      activation=tf.nn.elu,
                      kernel_initializer=he_init,
                      kernel_regularizer=l2_regularizer)

hidden1 = dense_layer(X, n_hidden1)
hidden2 = dense_layer(hidden1, n_hidden2)
hidden3 = dense_layer(hidden2, n_hidden3)
outputs = dense_layer(hidden3, n_outputs, activation=None)
print(hidden2)

Tensor("dense_3/Elu:0", shape=(?, 3), dtype=float32)


In [60]:
reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([reconstruction_loss] + reg_losses)

optimizer = tf.train.AdamOptimizer(0.01)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [61]:
saver = tf.train.Saver()

In [62]:
n_iterations = 10000

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={X: returns_arr_scaled})
    writer = tf.summary.FileWriter('./temp', sess.graph)
    outputs_val = outputs.eval(feed_dict={X: returns_arr_scaled})
    print(outputs_val)

[[ 0.97946882  0.992661    1.02458549 ...,  0.81081712  0.62604719
   0.44461855]
 [ 0.67783952  0.68872225  0.7153101  ...,  0.7553196   0.60763907
   0.46261394]
 [-0.3143566  -0.31106633 -0.30200922 ...,  0.57286525  0.54724216
   0.52199876]
 ..., 
 [-0.21983463 -0.21632957 -0.19760746 ...,  0.16304553  0.2231558
   0.27080807]
 [ 0.98651946  0.99818194  1.05489111 ..., -0.50786716 -0.39281976
  -0.31472552]
 [ 1.2012881   1.2143501   1.27759659 ..., -0.62946761 -0.50494736
  -0.42162651]]


In [63]:
import numpy as np
data=scaler.inverse_transform(outputs_val,copy=None)
#recovered
#data
orig=np.linalg.norm(outputs_val,ord=1)
DiffData=data-returns_arr
NewNorm=np.linalg.norm(DiffData,ord=1)
error=NewNorm/orig 
error
#recovering the orignal dat

0.0001994338265556331